# Convert DNBs Additional Validation Rules to Patterns

DNBs additional validation rules are available in the 'solvency2-rules' subfolder of the repository.  
The formulas in this file use a specific syntax, this notebook converts this syntax to a syntax that can be interpreted by Python.  
The resulting formulas are called 'patterns'.

## Import packages

In [ ]:
import pandas as pd  # dataframes
from os.path import join # some os dependent functionality
# from dqr import Evaluator  # conversion from 'rules' to expressions for the data-patterns packages

## General parameters

In [ ]:
# Location and name of the file with the additional rules:
RULES_PATH = join('..', 'data', 'downloaded files')  
FILENAME_RULES = '2021_04-01_set_aanvullende_controleregels_solvency2.xlsx'

In [ ]:
# Location and names of files with all possible datapoints for QRS and ARS
DATAPOINTS_PATH = join('..', 'data', 'datapoints')
FILENAME_DATAPOINTS_QRS = 'QRS.csv'
FILENAME_DATAPOINTS_ARS = 'ARS.csv'

In [ ]:
# Input parameters:
PARAMETERS = {'decimal': 0}
# currently only 'decimal' is available which specifies tolerance during evaluation of patterns.
# decimal: 0 means tolerance = abs(1.5e-0) (= 1.5)

In [ ]:
# # We log to rules.log in the data/instances path
# logging.basicConfig(filename = join(INSTANCES_DATA_PATH, 'rules.log'),level = logging.INFO, 
#                     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Read file with all possible datapoints

We use a simplified taxonomy with all possible datapoints, located in the data/datapoints directory.  
The evaluator uses this taxonomy to generate the patterns.

In [ ]:
# Load files to dataframe:
df_datapoints_qrs = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_QRS), sep=";").fillna("")
df_datapoints_ars = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_ARS), sep=";").fillna("")

In [ ]:
df_datapoints_ars.head()

## Read DNBs Additional Validation Rules

DNBs additional validation rules are currently published as an Excel file on the DNB statistics website. We included the Excel file here in the project.

Here we read the Excel and perform some data cleaning.

In [ ]:
df_rules = pd.read_excel(join(RULES_PATH, FILENAME_RULES), header = 1, engine='openpyxl')
df_rules.drop_duplicates(inplace=True) #remove double lines
df_rules.fillna("", inplace = True)
df_rules = df_rules.set_index('ControleRegelType')

In [ ]:
df_rules

<> " " has to be converted to <> None

In [ ]:
df_rules['Formule'] = df_rules['Formule'].str.replace('" "','None')

Some rules check dates to be filled by > 0, this has to be changed to <> None

In [ ]:
list_of_rules = ['S.15.01_105',
                 'S.15.01_107',
                 'S.23.04_111',
                 'S.23.04_112',
                 'S.23.04_121',
                 'S.23.04_122',
                 'S.23.04_133',
                 'S.23.04_144',
                 'S.23.04_145', 
                 'S.30.01_105',
                 'S.30.01_106',
                 'S.30.01_117',
                 'S.30.01_118',
                 'S.30.03_102',
                 'S.30.03_103',
                 'S.36.01_106',
                 'S.36.02_106',
                 'S.36.02_108',
                 'S.36.03_104',
                 'S.10.01_115',
                 'S.15.01_106',
                 'S.15.01_108',
                 'S.23.04_127',
                 'S.23.04_128',
                 'S.23.04_137',
                 'S.23.04_148',
                 'S.23.04_149']

df_rules.loc[list_of_rules, 'Formule'] = df_rules.loc[list_of_rules, 'Formule'].str.replace("> 0",'<> None').str.replace(">0",'<> None')

The Excel file contains rules for different report-types. In the next step we filter out the rules for QRS and ARS respectively.

In [ ]:
df_rules_qrs = df_rules.copy()[(df_rules['Standaard'] == 'SOLVENCY') | (df_rules['Standaard'] == 'QRS')]
df_rules_ars = df_rules.copy()[(df_rules['Standaard'] == 'SOLVENCY') | (df_rules['Standaard'] == 'ARS')]

In [ ]:
df_rules_qrs.head()

In [ ]:
df_rules_ars.head()

## Convert the rules to patterns

The evaluator is a piece of Python code, which takes the Additional Validation Rules as input, and transforms it to expressions that can be interpreted by the data_patterns package (patterns).

In [ ]:
import pandas as pd
import numpy as np
import os
from os.path import join
import re
import sys
import data_patterns
import logging


class Evaluator:
    def __init__(self, df_rules, df_datapoints, parameters):

        self.entrypoint_templates = sorted(list(df_datapoints['tabelcode'].unique()))
        self.entrypoint_datapoints = sorted(list((df_datapoints['tabelcode'] + "," +
                                                  df_datapoints['rij'] + "," +
                                                  df_datapoints['kolom']
                                                  ).str.replace(",,", ",")))
        self.df_rules = self.pre_process_rules(df_rules)
        self.df_patterns = self.process_rules(df_datapoints, parameters)

    def datapoints2pandas(self, s):
        """Transform EVA2 datapoints to Python Pandas datapoints by making letters uppercase"""
        datapoints = []
        for item in list(set(re.findall(r'{(.*?)}', s))):
            datapoints.append(item.upper())
            s = s.replace(item, '"' + item.upper() + '"')
        s = self.preprocess_pattern(s)
        return s, datapoints

    def replace_and_or(self, s):
        """Replace and by & and or by |, but not within strings"""
        if re.search(r"(.*?)\"(.*?)\"(.*)", s) is None:  # input text does not contain strings
            s = s.replace("OR", "|")
            s = s.replace("AND", "&")
        for item in re.findall(r"(.*?)\"(.*?)\"(.*)", s):
            s = s.replace(item[0], item[0].replace("OR", "|"))
            s = s.replace(item[0], item[0].replace("AND", "&"))
            s = s.replace(item[2], self.replace_and_or(item[2]))
        return s

    def preprocess_pattern(self, pattern):

        pattern = pattern.replace("<>", "!=")
        pattern = pattern.replace("< >", "!=")  # the space between < and > should be deleted in EVA2
        pattern = pattern.replace(';', ",")  # this should be corrected in EVA2

        # pattern = pattern.replace('"', "'")
        pattern = self.replace_and_or(pattern)
        return pattern

    def make_pattern_expression(self, expression, name, parameters):
        """Make expressions for the miner"""
        parameters['solvency'] = True
        pandas_expressions = data_patterns.to_pandas_expressions(expression, {}, parameters, None)
        pattern = [[name, 0] + [expression] + [0, 0, 0] + ["DNB"] + [{}] + pandas_expressions + ["", "", ""]]
        return pattern

    def pre_process_rules(self, df_rules):
        """Transform rules so that we can evaluate them. Not all rules are fit to be evaluated"""
        logger = logging.getLogger(__name__)
        df_rules['datapoints'] = ''
        df_rules['datapoints'] = df_rules['datapoints'].astype('object')
        df_rules['templates'] = ''
        df_rules['templates'] = df_rules['templates'].astype('object')
        for row in df_rules.index:
            rule_original = df_rules.loc[row, 'Formule']
            if not isinstance(rule_original, str):
                logger.info("Rule " + row + ": " + "duplicate rule. ")
                rule_original = rule_original.values[0]
            rule_original, datapoints = self.datapoints2pandas(rule_original)
            df_rules.at[row, 'datapoints'] = datapoints
            df_rules.at[row, 'templates'] = list(set([datapoint[0:13].upper() for datapoint in datapoints]))
            df_rules.loc[row, 'Formule_input'] = rule_original

        df_rules['error'] = ''  # error message
        df_rules['n_patterns'] = 0  # number of patterns derived from rules

        return df_rules

    def process_rule(self, pre_expression, name, datapoints, expansion_dict, df_datapoints, parameters):
        """Some rules have multiple rows or columns. This function makes all the expressions with every row/column"""
        expressions = []
        invalid_expressions = ""
        if expansion_dict:
            if datapoints[0] in expansion_dict.keys():
                zero = datapoints[0]
            else:
                zero = datapoints[1]
            for i in range(len(expansion_dict[zero])):
                expression = pre_expression
                valid_expression = True
                for datapoint in datapoints:
                    if datapoint in expansion_dict.keys():
                        if len(df_datapoints[(df_datapoints['tabelcode'] == expansion_dict[datapoint][i][:13]) &
                                             (df_datapoints['rij'] == expansion_dict[datapoint][i][14:19].upper()) &
                                             (df_datapoints['kolom'] == expansion_dict[datapoint][i][20:26].upper())]) == 0:
                            valid_expression = False
                        expression = expression.replace(datapoint, expansion_dict[datapoint][i])
                if valid_expression:
                    expressions.extend(self.make_pattern_expression(expression, name, parameters))
                else:
                    if invalid_expressions == "":
                        invalid_expressions = invalid_expressions + "(" + expression + ")"
                    else:
                        invalid_expressions = invalid_expressions + ", (" + expression + ")"
        else:
            expressions.extend(self.make_pattern_expression(pre_expression, name, parameters))

        return expressions, invalid_expressions

    def process_rules(self, df_datapoints, parameters):
        """Evaluate all rules and stores the result in df_rules"""
        logger = logging.getLogger(__name__)
        rules_expressions = []
        for idx in range(len(self.df_rules.index)):
            row = self.df_rules.index[idx]
            rule_original = self.df_rules.loc[row, 'Formule_input']
            if ("SUBSTR" not in rule_original) and \
               ("NOT CONTAINS" not in rule_original) and \
               ("CONTAINS" not in rule_original) and \
               ("SUM" not in rule_original) and \
               ("= ." not in rule_original) and \
               ("=." not in rule_original):
                rule_name = self.df_rules.index[idx]
                datapoints = self.df_rules.loc[row, 'datapoints'].copy()
                templates = self.df_rules.loc[row, 'templates']
                self.df_rules['Rijen'] = self.df_rules['Rijen'].astype(str)
                self.df_rules['Kolommen'] = self.df_rules['Kolommen'].astype(str)
                row_range = self.df_rules.loc[row, 'Rijen'].replace("(", "").replace(")", "").replace(",", ";").split(";")
                column_range = self.df_rules.loc[row, 'Kolommen'].replace("(", "").replace(")", "").replace(",", ";").split(";")
                # are the templates in the rule in the instance?
                templates_not_found = []
                for template in templates:
                    if template not in self.entrypoint_templates:
                        templates_not_found.append(template)

                if templates_not_found == []:
                    datapoints_not_found = []
                    expansion_dict = {}
                    # are the datapoints in the rule in the instance?
                    for datapoint in datapoints:
                        if datapoint not in self.entrypoint_datapoints:  # if datapoint is not there, see if we need to add rows or columns
                            new_list = []
                            if datapoint[14] == "C" and (len(row_range) > 1 or row_range[0].upper() == "ALL"):
                                if len(row_range) == 1 and row_range[0].upper() == "ALL":
                                    for col in self.entrypoint_datapoints:
                                        reg = re.search(datapoint[0:14] + "R....," + datapoint[14:],col)  # do for all rows if necessary
                                        if reg:
                                            new_list.append(reg.group(0))
                                else:
                                    rows = []
                                    for r in row_range:
                                        if len(r) - len(r.replace("-", "")) == 1:  # range
                                            low, high = r.split("-")
                                            rows.extend(list(df_datapoints[(df_datapoints['tabelcode'] == datapoint[0:13]) &
                                                                           (df_datapoints['kolom'] == datapoint[14:]) &
                                                                           (df_datapoints['rij'].str[-4:] >= low) &
                                                                           (df_datapoints['rij'].str[-4:] <= high)
                                                                           ].rij))
                                        else:
                                            if r.upper()[0] == 'R':
                                                rows.extend([r.upper()])
                                            else:
                                                rows.extend([('R' + r)])
                                    for r in rows:
                                        new_list.append(datapoint[0:14] + r + "," + datapoint[14:])
                            if datapoint[14] == "R" and(len(column_range) > 1 or column_range[0].upper() == "ALL"):
                                if len(column_range) == 1 and column_range[0].upper() == "ALL":
                                    for col in self.entrypoint_datapoints:
                                        reg = re.search(datapoint + ",C....", col)  # do for all columns if necessary
                                        if reg:
                                            new_list.append(reg.group(0))
                                else:
                                    cols = []
                                    for c in column_range:
                                        if len(c) - len(c.replace("-", "")) == 1:  # range
                                            low, high = c.split("-")
                                            cols.extend(list(df_datapoints[(df_datapoints['tabelcode'] == datapoint[0:13]) &
                                                                           (df_datapoints['rij'] == datapoint[14:]) &
                                                                           (df_datapoints['kolom'].str[-4:] >= low) &
                                                                           (df_datapoints['kolom'].str[-4:] <= high)
                                                                           ].rij))
                                        else:
                                            if c.upper()[0] == 'C':
                                                cols.extend([c.upper()])
                                            else:
                                                cols.extend([('C' + c)])
                                    for c in cols:
                                        new_list.append(datapoint + "," + c)
                            if new_list != []:
                                expansion_dict[datapoint] = new_list
                            else:
                                datapoints_not_found.append(datapoint)
                    if datapoints_not_found == []:
                        rule_expressions, invalid_expressions = self.process_rule(rule_original, rule_name, datapoints, expansion_dict, df_datapoints, parameters)
                        rules_expressions.extend(rule_expressions)
                        if invalid_expressions != "":
                            self.df_rules.loc[row, 'error'] = \
                                'Some expressions skipped due to invalid datapoint references: ' + invalid_expressions
                            logger.warning("Rule " + row + ", " + self.df_rules.loc[row, 'error'])
                        else:
                            self.df_rules.loc[row, 'error'] = ''
                        self.df_rules.loc[row, 'n_patterns'] = len(rule_expressions)
                        logger.info("Rule " + row + ", " + str(len(rule_expressions)) + " pattern(s) generated")
                    else:
                        # expression = rule_original
                        self.df_rules.loc[row, 'error'] = 'missing datapoint(s): ' + str(datapoints_not_found)
                        logger.warning("Rule " + row + ", " + self.df_rules.loc[row, 'error'])
                else:
                    # expression = rule_original
                    self.df_rules.loc[row, 'error'] = 'missing template(s): ' + str(templates_not_found)
                    logger.warning("Rule " + row + ", " + self.df_rules.loc[row, 'error'])

        df_patterns = pd.DataFrame(data = rules_expressions, columns = data_patterns.PATTERNS_COLUMNS)
        df_patterns.index.name = 'index'

        return df_patterns


In [ ]:
evaluator_qrs = Evaluator(df_rules_qrs, df_datapoints_qrs, PARAMETERS)

In [ ]:
evaluator_qrs.df_patterns

In [ ]:
df_rules_ars_1 = df_rules_ars.iloc[0:400, :]

evaluator_ars = Evaluator(df_rules_ars, df_datapoints_ars, PARAMETERS)

In [ ]:
evaluator_ars.df_patterns

In [ ]:
evaluator_qrs.df_patterns.head()

In [ ]:
evaluator_ars.df_patterns.head()

## Export patterns to rules folder

In [ ]:
evaluator_qrs.df_patterns.to_excel(join('..', 'solvency2-rules', "qrs_patterns_additional_rules.xlsx"))

In [ ]:
evaluator_ars.df_patterns.to_excel(join('..', 'solvency2-rules', "ars_patterns_additional_rules.xlsx"))